In [ ]:
import glob
import numpy as np
import pandas as pd
from astropy.time import Time
import cdflib
import datetime
import spacepy.datamodel as dm
from astropy.table import Table
import math
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
import tqdm


def addDatetimes(df, dftype):
    datetimes = []

    if dftype == 'mlh':
        for i in iter(range(0, len(df))):
            datetimes.append(datetime.datetime(df['YEAR'][i],
                                               df['MONTH'][i],
                                               df['DAY'][i],
                                               df['HOUR'][i],
                                               df['MIN'][i],
                                               df['SEC'][i]))
    if dftype == 'grf':
        for i in iter(range(0, len(df))):
            datetimes.append(datetime.datetime(df['yy'][i],
                                               df['mo'][i],
                                               df['dd'][i],
                                               df['hh'][i],
                                               df['mm'][i],
                                               df['ss'][i]))

    df['datetimes'] = datetimes
    df.index = df.datetimes

    return df

def earth_radius(B):
    B = math.radians(B)  # converting into radians
    a = 6378.137  # Radius at sea level at equator
    b = 6356.752  # Radius at poles
    c = (a ** 2 * math.cos(B)) ** 2
    d = (b ** 2 * math.sin(B)) ** 2
    e = (a * math.cos(B)) ** 2
    f = (b * math.sin(B)) ** 2
    R = math.sqrt((c + d) / (e + f))
    return R

def objective(x, a, b):
    return  a*np.exp(b*x)

def find_nearest_idx(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

spatial_window = 5
temporal_window = 15
altitude_window = 100

radar_list = glob.glob('../data/external/Madrigal/2018-2021/*/*.hdf5')
radar_list.sort()

gf_list = glob.glob('../data/interim/GRACEFO/KBRNE_relative_Chao/*/*.dat')
gf_list.sort()
#
# headerlist = ['CDF Epoch', 'GPS', 'Latitude', 'Longitude', 'Radius', 'Latitude_QD', 'Longitude_QD',
#               'MLT', 'GRACE_1_Position', 'GRACE_2_Position', 'Iono_Corr', 'Distance', 'Relative_Hor_TEC', 'Relative_Ne']

gf_list_datetimes = []
for gf_file in gf_list:
    gf = pd.read_csv(gf_file, sep='\s+')
    gf_list_datetimes.append((datetime.datetime(gf['yy'][0],gf['mo'][0],gf['dd'][0])).strftime('%Y-%m-%d'))


def find_conjunctions(radar_file, gf_list, gf_list_datetimes):
    radar_select_0 = []
    gf_select_0 = []
    radar_select_1 = []
    gf_select_1 = []
    radar_select_2 = []
    gf_select_2 = []

    radar_metadata = []


    radar = (Table.read(radar_file, path='Data/Table Layout')).to_pandas()
    radar_date = datetime.datetime(int(radar.year[0]), int(radar.month[0]), int(radar.day[0])).strftime('%Y-%m-%d')

    date, idx_gf, idx_radar = np.intersect1d(gf_list_datetimes, radar_date, return_indices=True)

    if len(idx_radar) > 0:

        gf_file = gf_list[idx_gf[0]]
        gf = pd.read_csv(gf_file, sep='\s+')
        gf = addDatetimes(gf,'grf')

        radar_metadata = (Table.read(radar_file, path='Metadata/Experiment Parameters')).to_pandas()

        names = np.array([x.decode() for x in radar_metadata['name']])
        values = np.array([x.decode() for x in radar_metadata['value']])

        radar_metadata = pd.DataFrame({'name': names,
                                       'value': values})
        radar_lat = float(radar_metadata[radar_metadata['name'] == 'instrument latitude']['value'].values[0])
        radar_lon = float(radar_metadata[radar_metadata['name'] == 'instrument longitude']['value'].values[0])
        radar_lon = (((radar_lon + 180) % 360) - 180)
        radar['dates'] = Time(radar[['ut1_unix', 'ut2_unix']].mean(axis=1), format='unix').datetime

        gf = gf[((gf['Latitude'] <= radar_lat + spatial_window) & (gf['Latitude'] >= radar_lat - spatial_window))]
        gf = gf[((gf['Longitude'] <= radar_lon + spatial_window) & (gf['Longitude'] >= radar_lon - spatial_window))]

        # gf = gf[gf['Ne'] >= 0]
        if len(gf) > 0:

            gf_pass = gf[gf.diff(axis=0)['dates'] > datetime.timedelta(minutes=temporal_window)]

            if len(gf_pass) == 0:  # only one passage

                time_window_i = gf['dates'][gf.index.min()] - datetime.timedelta(minutes=temporal_window)
                time_window_f = gf['dates'][gf.index.max()] + datetime.timedelta(minutes=temporal_window)

                radar['dates'] = Time(radar[['ut1_unix', 'ut2_unix']].mean(axis=1), format='unix').datetime

                radar_select_0 = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
                gf_select_0 = gf[(gf['dates'] >= time_window_i) & (gf['dates'] <= time_window_f)]

                if len(radar_select_0) > 0:
                    Re = earth_radius(radar_lat)
                    altitude = np.mean(gf_select_0['Radius'] * 0.001 - Re)
                    # print(altitude, np.mean(radar_select_0['gdalt']))

                    altitude_window_i = altitude - altitude_window
                    altitude_window_f = altitude + altitude_window

                    radar_select_0 = radar_select_0[
                        (radar_select_0['gdalt'] >= altitude_window_i) & (radar_select_0['gdalt'] <= altitude_window_f)]


            if len(gf_pass) == 1:  # two passages

                pass_idx = gf[gf['dates'] == gf_pass['dates'][gf_pass.index.min()]].index[0]

                # first one

                # define time window
                time_window_i = gf['dates'][gf.index.min()] - datetime.timedelta(minutes=15)
                time_window_f = gf['dates'][
                                    gf.index[np.argwhere(gf.index == pass_idx)[0][0] - 1]] + datetime.timedelta(
                    minutes=15)

                radar_select_1 = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
                gf_select_1 = gf[(gf['dates'] >= time_window_i) & (gf['dates'] <= time_window_f)]

                if len(radar_select_1) > 0:
                    Re = earth_radius(radar_lat)
                    altitude = np.mean(gf_select_1['Radius'] * 0.001 - Re)
                    # print(altitude, np.mean(radar_select_1['gdalt']))

                    altitude_window_i = altitude - altitude_window
                    altitude_window_f = altitude + altitude_window

                    radar_select_1 = radar_select_1[
                        (radar_select_1['gdalt'] >= altitude_window_i) & (radar_select_1['gdalt'] <= altitude_window_f)]


                # second one

                # define time window
                time_window_i = gf['dates'][gf.index[np.argwhere(gf.index == pass_idx)[0][0]]] - datetime.timedelta(
                    minutes=15)
                time_window_f = gf['dates'][gf.index.max()] + datetime.timedelta(minutes=15)

                radar_select_2 = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
                gf_select_2 = gf[(gf['dates'] >= time_window_i) & (gf['dates'] <= time_window_f)]

                if len(radar_select_2) > 0:
                    Re = earth_radius(radar_lat)
                    altitude = np.mean(gf_select_2['Radius'] * 0.001 - Re) # 0.001 convert m to km
                    # print(altitude, np.mean(radar_select_2['gdalt']))

                    altitude_window_i = altitude - altitude_window
                    altitude_window_f = altitude + altitude_window

                    radar_select_2 = radar_select_2[
                        (radar_select_2['gdalt'] >= altitude_window_i) & (radar_select_2['gdalt'] <= altitude_window_f)]

    del radar
    del gf

    return radar_select_0,gf_select_0,radar_select_1,gf_select_1,radar_select_2,gf_select_2,radar_metadata, Re

# radar_metadata[radar_metadata['name'] == 'Cedar file name']['value']
# gf_file.split('/')[-1]

def calc_conjunctions(radar_select, gf_select,Re):
    radar_select['nel'] = np.log10(radar_select['ne'])
    gf_select['Relative_Nel'] = np.log10(gf_select['Relative_Ne'])

    radar_mean = radar_select.groupby(['gdalt']).mean()

    gf_alt = np.mean(gf_select['Radius'] * 0.001 - Re)
    gf_nel = np.mean(gf_select['Relative_Nel']) + np.abs(np.min((gf_select['Relative_Nel'])))


    x = radar_mean['nel'].values
    y = radar_mean.index.values

    # curve fit
    popt, _ = curve_fit(objective, x, y, maxfev=10000)
    a, b = popt
    #         print(popt)

    # define a sequence of inputs between the smallest and largest known inputs
    x_line = np.arange(min(x), max(x), 10e-6)
    # calculate the output for the range
    y_line = objective(x_line, a, b)

    idx = find_nearest_idx(y_line,gf_alt)
    radar_alt = y_line[idx]
    radar_nel = x_line[idx]
    plt.scatter(gf_nel,radar_nel)
    return radar_nel,gf_nel

fig, ax = plt.subplots(figsize=(10,10))
plt.plot([0,15],[0,15], color='k')
plt.xlim(0,15)
plt.ylim(0,15)


radars = glob.glob('../data/external/Madrigal/2018-2021/*')
for r in iter(range(2,3)):
    print('------')
    print(r,radars[r])
    radar_list = (glob.glob('../data/external/Madrigal/2018-2021/' + radars[r].split('/')[-1] + '/*'))
    names = []
    count = 0

    for i in iter(range(0, len(radar_list))):

        try:
            radar_select_0,gf_select_0,radar_select_1,gf_select_1,radar_select_2,gf_select_2,radar_metadata, Re = find_conjunctions(radar_file= radar_list[i], gf_list = gf_list, gf_list_datetimes = gf_list_datetimes)

            if (len(radar_select_0) > 0) & (len(gf_select_0) > 0):
                count = count + 1
                # print('hi')
            #     # print('0 GF passage ', gf_select_0['dates'][gf_select_0.index == gf_select_0.index.min()].values, '->', gf_select_0['dates'][gf_select_0.index == gf_select_1.index.max()].values)
            #     # print('gf : ', len(gf_select_0))
            #     # print('radar : ',len(np.unique(radar_select_0['recno'])))
            #     # print('------')
            #     # print(radar_metadata[radar_metadata['name'] == 'instrument']['value'][0])
                radar_nel_0,gf_nel_0 = calc_conjunctions(radar_select_0, gf_select_0, Re)
            #     # n = n+1
            #
            if (len(radar_select_1) > 0) & (len(gf_select_1) > 0):
                count = count + 1
                # print('hello')
            #     # print('1 GF passage ', gf_select_1['dates'][gf_select_1.index == gf_select_1.index.min()].values, '->', gf_select_1['dates'][gf_select_1.index == gf_select_1.index.max()].values)
            #     # print('gf : ', len(gf_select_1))
            #     # print('radar : ',len(np.unique(radar_select_1['recno'])))
            #     # print('------')
            #     # print(radar_metadata[radar_metadata['name'] == 'instrument']['value'][0])
                radar_nel_1,gf_nel_1 =calc_conjunctions(radar_select_1, gf_select_1, Re)
            #     # n = n + 1
            #
            if (len(radar_select_2) > 0) & (len(gf_select_2) > 0):
                count = count + 1
                # print('good morning')
            #     # print('2 GF passage ', gf_select_2['dates'][gf_select_2.index == gf_select_2.index.min()].values, '->', gf_select_2['dates'][gf_select_2.index == gf_select_2.index.max()].values)
            #     # print('gf : ', len(gf_select_2))
            #     # print('radar : ', len(np.unique(radar_select_2['recno'])))
            #     # print('------')
            #     # print(radar_metadata[radar_metadata['name'] == 'instrument']['value'][0])
                radar_nel_2,gf_nel_2 =calc_conjunctions(radar_select_2, gf_select_2, Re)
            #             # n = n + 1

        except:
            pass

    print(count)
    print(len(radar_list))

plt.show()
    # names = pd.DataFrame(data={"name": names})
    # names.to_csv('{r}.csv'.format(r = r))



------
2 ../data/external/Madrigal/2018-2021/MillstoneHillISRadar
